In [25]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from PIL import Image
import torch.optim as optim


In [26]:
def get_pickle_files(folder_path):
    """
    Get all the .pickle files from a folder.

    Args:
    - folder_path (str): Path to the folder containing .pickle files

    Returns:
    - List of .pickle files
    """
    pickle_files = [file for file in os.listdir(folder_path) if file.endswith('.pickle')]
    return pickle_files

In [27]:
def convert_to_3d_tensors(radar_return, target_size=(100, 50)):
    """
    Convert radar_return values to 3D torch tensors.

    Args:
    - radar_return (ndarray): 2D matrix consisting of complex numbers

    Returns:
    - 3D torch tensor with shape (width, height, channels)
    """
    img = Image.fromarray(radar_return)
    # Resize the image while preserving aspect ratio
    img = img.resize(target_size, resample=Image.BILINEAR)
    # Convert the PIL Image to NumPy array
    resized_img = np.array(img)
    # Split real and imaginary parts
    real_part = resized_img.real
    imag_part = resized_img.imag
    # Stack real and imaginary parts to create a 3D array with two channels
    array = np.stack((real_part, imag_part), axis=0)
    return array

In [28]:
def process_pickle_files(folder_path):
    """
    Process all .pickle files in a folder.

    Args:
    - folder_path (str): Path to the folder containing .pickle files

    Returns:
    - DataFrame containing all the samples
    """
    # Get .pickle files
    pickle_files = get_pickle_files(folder_path)
    dataframes = []
    for file_name in pickle_files:
        with open(os.path.join(folder_path, file_name), 'rb') as f:
            file_data = pickle.load(f)
            radar_return = file_data['radar_return']
            object_id = file_data['object_id']
            concatenated_radar = convert_to_3d_tensors(radar_return).astype('float32')
            # Create a DataFrame with concatenated radar and object_id
            df = pd.DataFrame({'radar_return': [concatenated_radar], 'object_id': [object_id]})
            # Append the DataFrame to the list
            dataframes.append(df)

    # Concatenate the list of DataFrames into a single DataFrame
    df = pd.concat(dataframes, ignore_index=True)
    return df

In [29]:
# Example usage
folder_path = 'Overfit_data'
df = process_pickle_files(folder_path)

# Access a specific row (e.g., the first row)
sample_row = df.iloc[2]

# Access the 'radar_return' column of the sample row
sample_image = sample_row['radar_return']

# Display shape and values
print("Shape of the sample image:", sample_image.shape)
print("Sample image values:")
print(sample_image)

In [30]:

def plot_channels(tensor):
    """
    Plot each channel of a 3D tensor as grayscale images.

    Args:
    - tensor (ndarray): 3D numpy array with shape (channels, height, width)
    """
    num_channels = tensor.shape[0]
    fig, axes = plt.subplots(1, num_channels, figsize=(5*num_channels, 5))
    
    for i in range(num_channels):
        ax = axes[i] if num_channels > 1 else axes
        ax.imshow(tensor[i], cmap='gray')
        ax.set_title(f'Channel {i+1}')
        ax.axis('off')
    
    plt.show()


plot_channels(sample_image)

In [31]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Define the convolutional layers
        self.conv1 = nn.Conv2d(in_channels=2, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        # Define the max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Define the fully connected layers
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)  # Output size is 10 for classification (assuming 10 classes)

    def forward(self, x):
        # Apply convolutional layers with ReLU activation function and max pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # Flatten the input for the fully connected layers
        x = x.view(-1, 32 * 16 * 16)
        # Apply fully connected layers with ReLU activation function
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # Apply the final fully connected layer without activation function
        x = self.fc3(x)
        return x

In [36]:
class RadarDataset(Dataset):
    def __init__(self, df, target_size=(100, 50)):
        self.df = df
        self.target_size = target_size

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        radar_return = self.df.iloc[idx]['radar_return']
        object_id = self.df.iloc[idx]['object_id']
        # Convert radar return to 3D tensor
        radar_return = self.convert_to_3d_tensors(radar_return)
        return radar_return, object_id

    def convert_to_3d_tensors(self, radar_return):
        """
        Convert radar_return values to 3D tensor.

        Args:
        - radar_return (ndarray): 2D matrix consisting of complex numbers

        Returns:
        - 3D tensor with shape (channels, height, width)
        """
        # Convert radar_return to PIL Image
        img = Image.fromarray(radar_return)
        # Resize the image while preserving aspect ratio using BILINEAR filter
        img = img.resize(self.target_size, resample=Image.BILINEAR)
        # Convert the resized image to NumPy array
        resized_img = np.array(img)
        # Split real and imaginary parts
        real_part = resized_img.real
        imag_part = resized_img.imag
        # Stack real and imaginary parts to create a 3D array with two channels
        array = np.stack((real_part, imag_part), axis=0)
        # Convert the array to PyTorch tensor
        tensor = torch.tensor(array, dtype=torch.float32)
        return tensor

In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
# Print the architecture
print(model)


SimpleCNN(
  (conv1): Conv2d(2, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)


In [38]:
dataset = RadarDataset(df)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [39]:
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {running_loss/100}')
            running_loss = 0.0

print('Finished Training')

TypeError: Cannot handle this data type: (1, 1, 100), <f4